In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)

from src.extractor_utils import (predict, getPrediction)

from utils.make_submission import *

In [2]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [8]:
use_gpu = torch.cuda.is_available()
use_DataParalel= True # Need some improvement on model's to work properly, basically it has to wrap all models block:
use_CPU= False         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') #{} '.format(device_id))
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using DataParalel in all 4 GPUS


In [9]:
if use_gpu:
    !free -h
    !nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         51G         99G        566M         69G        166G
Swap:            0B          0B          0B
Sat Nov 25 11:51:13 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   63C    P0   129W / 149W |   6705MiB / 11439MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0


# Choose dataset 

In [10]:
path2train = "/mnt/home/e209440/data/train" 
path2test = "/mnt/home/e209440/data/test" 

# Images transformation

In [11]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}        

# Creating  Dataset & Dataloader

In [12]:
batch_size = 32
use_only = 0.1 # Use only is the percentage of the full dataset

In [13]:
dsets = {
    'train': KaggleSafeDriverDataset(path2train, transforms=data_transforms['train'],use_only=use_only),
    'valid': KaggleSafeDriverDataset(path2train, transforms=data_transforms['valid'],use_only=use_only, is_val=True, val_size=0.2),
    'test':  KaggleSafeDriverDataset(path2test, transforms=data_transforms['valid'],use_only=use_only, is_test=True),
}

In [14]:
dset_loaders = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= True)

In [15]:
dset_sizes = {x: len(dsets[x]) for x in ['train','valid', 'test']} 
dset_classes = len(dsets['train'].y)
dset_classes, dset_sizes

# Dataset have much more samples than datatrain ***It comes from the test.zip****

(1794, {'test': 7972, 'train': 1794, 'valid': 448})

# Checking GPU usage

In [16]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         51G         99G        566M         69G        166G
Swap:            0B          0B          0B
Sat Nov 25 11:51:29 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   62C    P0   140W / 149W |   6705MiB / 11439MiB |     42%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

# Choosing Model

In [17]:
use_resnet = True
use_inception = False
use_denseNet = False

if use_resnet:
    print('Using ResNet model')
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')    
    model = MyDenseNet()

Using ResNet model


In [18]:
if use_gpu:
    print('Using GPU')# {}'.format(device_id))
    model.cuda()
    convnet = model.mrnc
    densenet = model.mrnd
    
elif use_DataParalel:
    print("Using all GPU's ")
    model = torch.nn.DataParallel(model)
    model.cuda()
    convnet = model.module.mrnc
    densenet = model.module.mrnd
else:
    convnet = model.mrnc
    densenet = model.mrnd

Using GPU


In [18]:
#print(model)

In [19]:
#extract features from images
convOutput_train = predict(dset_loaders['train'], convnet,use_gpu=use_gpu)
convOutput_valid = predict(dset_loaders['valid'], convnet,use_gpu=use_gpu)
convOutput_test = predict(dset_loaders['test'], convnet,use_gpu=use_gpu)

predict: 56/56 ok
predict: 13/13 ok
predict: 249/249 ok


In [20]:
print(convOutput_train['true'].size(), convOutput_train['pred'].size())
print(convOutput_valid['true'].size(), convOutput_valid['pred'].size())
print(convOutput_test['true'].size(), convOutput_test['pred'].size())

torch.Size([1794]) torch.Size([1794, 6400])
torch.Size([448]) torch.Size([448, 6400])
torch.Size([7972]) torch.Size([7972, 6400])


In [21]:
print(convOutput_train['true'].type(), convOutput_train['pred'].type())

torch.cuda.LongTensor torch.cuda.FloatTensor


In [23]:
conv_dset ={
    'train': TensorDataset(convOutput_train['pred'], convOutput_train['true']),
    'valid': TensorDataset(convOutput_valid['pred'], convOutput_valid['true']),
    'test': TensorDataset(convOutput_test['pred'], convOutput_test['true'])
} 

In [24]:
dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [25]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

{'test': 250, 'train': 57, 'valid': 14}

In [26]:
def compute_accuracy(logits, labels):
    _, preds = torch.max(logits.data, 1)
    return torch.sum(preds == labels.data) / labels.size(0)

In [27]:
path2saveModel = '/mnt/home/r120084/project/models/distractdriver'
#path2saveModel = '/mnt/home/e209440/models/ResNetDistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=1)

In [28]:
loss_fn = nn.CrossEntropyLoss()
num_epochs = 50

optimizer =  optim.Adam(densenet.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)
metrics = dict(acc=compute_accuracy)

params = {'model' : densenet, 
    'criterion': loss_fn,  
    'metrics': metrics, 
    'optimizer': optimizer, 
    'callbacks': [savebest,  ptt.PrintCallback()] #ptt.PlotCallback(),
}

In [29]:
trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,**params)

TypeError: __init__() got an unexpected keyword argument 'metrics'

In [29]:
trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

Start training for 50 epochs
  1:  25.2s   T: 1.66269   V: 0.55486 best
  2:  12.3s   T: 0.39758   V: 0.32263 best
  3:  12.5s   T: 0.23267   V: 0.60291 
  4:  12.4s   T: 0.19890   V: 0.24328 best
  5:  12.4s   T: 0.14350   V: 0.25054 
  6:  12.4s   T: 0.14992   V: 0.33450 
  7:  12.6s   T: 0.09498   V: 0.28285 
  8:  12.5s   T: 0.12788   V: 0.18683 best
  9:  12.1s   T: 0.10371   V: 0.31989 
 10:  12.4s   T: 0.09722   V: 0.27857 
 11:  12.1s   T: 0.08032   V: 0.05275 best
 12:  12.2s   T: 0.07264   V: 0.04699 best
 13:  12.2s   T: 0.06474   V: 0.35075 
 14:  10.2s   T: 0.08937   V: 0.04275 best
 15:   9.7s   T: 0.08375   V: 0.02636 best
 16:  10.5s   T: 0.10195   V: 0.03329 
 17:  10.7s   T: 0.05808   V: 0.03862 
 18:  10.7s   T: 0.05387   V: 0.06939 
 19:  10.8s   T: 0.04675   V: 0.15205 
 20:  10.5s   T: 0.04743   V: 0.18609 
 21:  10.6s   T: 0.08496   V: 0.03323 
 22:  11.3s   T: 0.02672   V: 0.03459 
 23:  12.2s   T: 0.03716   V: 0.15495 
 24:  12.3s   T: 0.05447   V: 0.02202 best

# Loading best epoch

In [41]:
trainer.load_state(path2saveModel)
densenet = trainer.model

In [31]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         52G        102G        571M         65G        165G
Swap:            0B          0B          0B
Sat Nov 25 10:43:47 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   65C    P0   139W / 149W |   8688MiB / 11439MiB |     94%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

In [32]:
#plot_metrics(trainer)

In [33]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

evaluate: 560/560 ok
evaluate: 140/140 ok


In [34]:
train_eval, valid_eval

({'losses': 0.00108448752059527}, {'losses': 0.01567110304955815})

In [ ]:
#get output of the trained model (only denset)
result_train = predict(dset_loaders_convnet['train'], densenet,use_gpu=use_gpu)
result_valid = predict(dset_loaders_convnet['valid'], densenet,use_gpu=use_gpu)
result_test = predict(dset_loaders_convnet['test'], densenet,use_gpu=use_gpu)

In [ ]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [ ]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [ ]:
print('Train: ', correct_train, '/', len(conv_dset['train']) )
print('Valid: ', correct_valid, '/', len(conv_dset['valid']) )

In [ ]:
#visualize_predictions(dsets['valid'], result_valid, correct_pred = True)

In [ ]:
#visualize_predictions(dsets['valid'], result_valid, correct_pred = False)

In [ ]:
#plot_cm_train_valid(result_train,result_valid)

In [ ]:
#plot_layers_weight(dsets,img_width=img_width, img_height=img_height,conv_model = model.mrnc,use_gpu=use_gpu)

# Make submission of the Test set

In [ ]:
create_submission(result_test, 'distracted_driver')